In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sys
print (sys.version)

2.7.12 |Anaconda 4.2.0 (64-bit)| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]


In [15]:
import struct;print struct.calcsize("P")*8

64


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import pyodbc
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
%matplotlib inline
from matplotlib import pyplot as plt
print pyodbc.dataSources()

{'ECL_CODA11': 'Oracle in instantclient_11_2', 'NZSQL': 'NetezzaSQL', 'IST_CODA11': 'Oracle in instantclient_11_2', 'BO_Lookup': 'Oracle in instantclient_11_2', 'Excel Files': 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'dBASE Files': 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)', 'MS Access Database': 'Microsoft Access Driver (*.mdb, *.accdb)'}


C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import cx_Oracle

In [18]:
conn=pyodbc.connect("Driver={Oracle in instantclient_11_2};SERVER=10.71.90.0;DATABASE=ODSPEU1.ams;UID=djohnson;PWD=RhiBae01!",ansi=True)

Error: ('HY000', '[HY000] [Oracle][ODBC][Ora]ORA-12560: TNS:protocol adapter error\n (12560) (SQLDriverConnect); [HY000] [Oracle][ODBC][Ora]ORA-12560: TNS:protocol adapter error\n (12560)')

In [21]:
conn=cx_Oracle.connect('djohnson','RhiBae01!','ODSPEU1.ams')

In [22]:
ab=pd.read_sql('''
SELECT ab.merchant_reference
,ab.trading_merchant_id,ab.trading_merchant_name,ab.clearing_mcc_code
,ab.transaction_processing_date,ab.transaction_currency,ab.transaction_amount_trx_curr
,ab.interchange_rate_description,ab.issuer_bin
,ab.card_brand,ab.contract_id,ab.issuer_country,ab.clearing_country_name,

CASE WHEN ab.card_type_group LIKE '%Debit%' THEN 'Debit'
WHEN ab.card_type_group LIKE '%Credit%' THEN 'Credit'
WHEN ab.card_type_group LIKE '%Prepaid%' THEN 'Prepaid'
END AS "CREDIT_DEBIT",

CASE WHEN ab.card_type_group LIKE '%Consumer%' THEN 'Consumer'
WHEN ab.card_type_group LIKE '%Commercial%' THEN 'Commercial'
END AS "CARD_TYPE",

ABS(ab.scheme_fee_eur) AS SF

FROM fdwo.acquirer_aib_newgen_files ab
WHERE ab.transaction_processing_date = TO_DATE ('2017/12/13', 'yyyy/mm/dd')
AND ab.transaction_type='Sale'
'''
,conn)

In [35]:
ab['MERCHANT_REFERENCE']=ab['MERCHANT_REFERENCE'].astype(float)
ab['CLEARING_MCC_CODE']=ab['CLEARING_MCC_CODE'].astype(float)
ab['ISSUER_BIN']=ab['ISSUER_BIN'].astype(float)
ab['TRADING_MERCHANT_ID']=ab['TRADING_MERCHANT_ID'].astype(float)

In [36]:
ab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232507 entries, 0 to 232506
Data columns (total 16 columns):
MERCHANT_REFERENCE              232507 non-null float64
TRADING_MERCHANT_ID             232507 non-null float64
TRADING_MERCHANT_NAME           232507 non-null object
CLEARING_MCC_CODE               232507 non-null float64
TRANSACTION_PROCESSING_DATE     232507 non-null datetime64[ns]
TRANSACTION_CURRENCY            232507 non-null object
TRANSACTION_AMOUNT_TRX_CURR     232507 non-null float64
INTERCHANGE_RATE_DESCRIPTION    232507 non-null object
ISSUER_BIN                      232507 non-null float64
CARD_BRAND                      232507 non-null object
CONTRACT_ID                     232507 non-null object
ISSUER_COUNTRY                  232507 non-null object
CLEARING_COUNTRY_NAME           232507 non-null object
CREDIT_DEBIT                    226348 non-null object
CARD_TYPE                       232507 non-null object
SF                              232507 non-null flo

In [25]:
ab_sf=pd.read_csv('d:\\djohnson\\Desktop\\GlobalCollect_DailySFProcessing20171214.csv')

In [26]:
ab_sf=ab_sf[['merchantreference','DepositMerchantId','Clearing_MCC','Transaction_ProcessingDate','Currency_Transaction_Code','PRSAmount_TransactionCurrency'
             ,'InterchangeRate_Description','Issuer_BIN','SchemeFee_Percent','SchemeFee_Amount','SchemeFee_Amount_Minimum']]

In [51]:
ab_sf=ab_sf.rename(columns={'merchantreference':'MERCHANT_REFERENCE'
                            ,'Clearing_MCC':'CLEARING_MCC_CODE'
                            ,'DepositMerchantId':'TRADING_MERCHANT_ID'
                            ,'Transaction_ProcessingDate':'TRANSACTION_PROCESSING_DATE'
                            ,'InterchangeRate_Description':'INTERCHANGE_RATE_DESCRIPTION'
                            ,'Issuer_BIN':'ISSUER_BIN'
                            ,'PRSAmount_TransactionCurrency':'TRANSACTION_AMOUNT_TRX_CURR'
                            ,'Currency_Transaction_Code':'TRANSACTION_CURRENCY'})

In [54]:
ab_sf['TRANSACTION_PROCESSING_DATE']=pd.to_datetime(ab_sf['TRANSACTION_PROCESSING_DATE'])

In [55]:
ab_sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244503 entries, 0 to 244502
Data columns (total 11 columns):
MERCHANT_REFERENCE              244503 non-null int64
TRADING_MERCHANT_ID             244503 non-null int64
CLEARING_MCC_CODE               244503 non-null int64
TRANSACTION_PROCESSING_DATE     244503 non-null datetime64[ns]
TRANSACTION_CURRENCY            244503 non-null object
TRANSACTION_AMOUNT_TRX_CURR     244503 non-null float64
INTERCHANGE_RATE_DESCRIPTION    244503 non-null object
ISSUER_BIN                      244503 non-null int64
SchemeFee_Percent               244503 non-null float64
SchemeFee_Amount                244503 non-null float64
SchemeFee_Amount_Minimum        244503 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(4), object(2)
memory usage: 20.5+ MB


In [57]:
sf_output=pd.merge(ab,ab_sf,on=['MERCHANT_REFERENCE','TRADING_MERCHANT_ID','CLEARING_MCC_CODE','TRANSACTION_PROCESSING_DATE'
                                ,'TRANSACTION_CURRENCY','TRANSACTION_AMOUNT_TRX_CURR','INTERCHANGE_RATE_DESCRIPTION','ISSUER_BIN'],how='left')

In [58]:
sf_output

,MERCHANT_REFERENCE,TRADING_MERCHANT_ID,TRADING_MERCHANT_NAME,CLEARING_MCC_CODE,TRANSACTION_PROCESSING_DATE,TRANSACTION_CURRENCY,TRANSACTION_AMOUNT_TRX_CURR,INTERCHANGE_RATE_DESCRIPTION,ISSUER_BIN,CARD_BRAND,CONTRACT_ID,ISSUER_COUNTRY,CLEARING_COUNTRY_NAME,CREDIT_DEBIT,CARD_TYPE,SF,SchemeFee_Percent,SchemeFee_Amount,SchemeFee_Amount_Minimum
0,2.253985e+11,11611044.0,8552963170Eldrscrols,7994.0,2017-12-13,SEK,189.00,BASE-CONSUMER DEBIT AND PREPAID EU REGULATED-P...,554501.0,MasterCard,8136,Sweden,United Kingdom,Debit,Consumer,0.048246,0.059,0.03,0.04
1,2.253985e+11,11611044.0,8552963170Eldrscrols,7994.0,2017-12-13,GBP,41.94,UK STD DB-UK STD DB,454742.0,Visa,8136,United Kingdom,United Kingdom,Debit,Consumer,0.024718,0.020,0.01,0.02
2,2.253985e+11,11611044.0,8552963170Eldrscrols,7994.0,2017-12-13,SEK,189.00,BASE-CONSUMER DEBIT AND PREPAID EU REGULATED-P...,554501.0,MasterCard,8136,Sweden,United Kingdom,Debit,Consumer,0.048246,0.059,0.03,0.04
3,2.253985e+11,11611044.0,8552963170Eldrscrols,7994.0,2017-12-13,SEK,309.00,BASE-CONSUMER DEBIT AND PREPAID EU REGULATED-P...,516815.0,MasterCard,8136,Sweden,United Kingdom,Debit,Consumer,0.052995,0.059,0.03,0.04
4,2.253985e+11,11618706.0,Madbid,7278.0,2017-12-13,GBP,27.99,UK STD DB-UK STD DB,476367.0,Visa,8750,United Kingdom,United Kingdom,Debit,Consumer,0.023144,0.020,0.01,0.02
5,2.253983e+11,11641262.0,www eastpak com,5691.0,2017-12-13,GBP,56.00,BASE-CONSUMER CREDIT EU REGULATED-POS_GENERAL,540438.0,MasterCard,5456,United Kingdom,Belgium,Credit,Consumer,0.068033,0.059,0.03,0.04
6,2.253983e+11,11641271.0,www kipling com,5691.0,2017-12-13,GBP,14.50,BASE-CONSUMER CREDIT EU REGULATED-POS_GENERAL,525682.0,MasterCard,5462,United Kingdom,Belgium,Credit,Consumer,0.046883,0.059,0.03,0.04
7,2.253983e+11,11641271.0,www kipling com,5691.0,2017-12-13,GBP,24.50,VE NON-SEC DB-VE NON-SEC DB,475127.0,Visa,5462,United Kingdom,Belgium,Debit,Consumer,0.022750,0.020,0.01,0.02
8,2.253981e+11,13112650.0,SEGA,5816.0,2017-12-13,GBP,37.99,VE NON-SEC CR-VE NON-SEC CR,402396.0,Visa,2046,United Kingdom,France,Credit,Consumer,0.028559,0.030,0.01,0.02
9,2.253981e+11,13116184.0,Valentino Rossi,5691.0,2017-12-13,GBP,37.81,FULL UCAF-CONSUMER CREDIT EU REGULATED-POS_GEN...,524679.0,MasterCard,2299,United Kingdom,Netherlands,Credit,Consumer,0.048628,0.039,0.03,0.04


In [ ]:
sf_output